# Notebook for brain voxels clustering

In this notebook we try to investigate networks of voxels in the brain when processing natural language and their potential meaning.
To do so, two approaches were considered:
    - fonctional clustering
    - semantic clustering

In [18]:
import warnings
warnings.simplefilter(action='ignore')

import os
import gc
import glob
import itertools
from tqdm import tqdm
from itertools import combinations
from joblib import Parallel, delayed

import umap
import scipy
import hdbscan
import nistats
import numpy as np
import pandas as pd
from sklearn import manifold
from sklearn.decomposition import PCA, FastICA
from sklearn.neighbors import kneighbors_graph
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.cluster import AgglomerativeClustering, KMeans

%matplotlib inline
import matplotlib
import seaborn as sns
import matplotlib.cm as cmx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import nibabel as nib
import nilearn
from nilearn.image import load_img, mean_img, index_img, threshold_img, math_img, smooth_img, new_img_like
from nilearn.input_data import NiftiMapsMasker, NiftiMasker, NiftiLabelsMasker, MultiNiftiMasker
from nistats.second_level_model import SecondLevelModel
from nistats.thresholding import map_threshold
from nilearn import plotting
from nilearn import datasets
from scipy.stats import norm
from nilearn.surface import vol_to_surf

import utils 
import reporting
from logger import Logger
from linguistics_info import load_surnames, load_syntactic_roi, load_language_roi

### Paths

In [3]:
language = 'english'
model_name = 'bert-base-cased_pre-7_1_post-0_norm-None'

In [4]:
PROJECT_PATH = f"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/"
OUTPUT_PATH = f"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/maps/{language}"
INPUT_PATH = f"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/stimuli-representations/{language}"
FMRIDATA_PATH = f"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/data/fMRI/{language}"
MASKER_PATH = f"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/ROI_masks/global_masker_95%_{language}"
ALL_MASKS_PATH = f"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/ROI_masks/"
SAVING_FOLDER = f"/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/{language}"
TMP_FOLDER = f"/home/ap259944/tmp"

### Maskers

In [5]:
logger = Logger(os.path.join(PROJECT_PATH, 'logs.txt'))

global_masker_50 = reporting.fetch_masker(f"{ALL_MASKS_PATH}/global_masker_{language}"
, language, FMRIDATA_PATH, INPUT_PATH, smoothing_fwhm=None, logger=logger)
global_masker_95 = reporting.fetch_masker(f"{ALL_MASKS_PATH}/global_masker_95%_{language}"
, language, FMRIDATA_PATH, INPUT_PATH, smoothing_fwhm=None, logger=logger)


original_masker = global_masker_50
new_masker = global_masker_95
original_masker.set_params(detrend=False, standardize=False)
new_masker.set_params(detrend=False, standardize=False)


NiftiMasker(detrend=False, dtype=None, high_pass=None, low_pass=None,
            mask_args=None,
            mask_img=<nibabel.nifti1.Nifti1Image object at 0x7f089c3adfd0>,
            mask_strategy='background', memory=Memory(location=None),
            memory_level=0, reports=True, sample_mask=None, sessions=None,
            smoothing_fwhm=None, standardize=False, t_r=None,
            target_affine=None, target_shape=None, verbose=0)

### Atlas

In [6]:
atlas_maps, labels = reporting.load_atlas() # load harvard-oxford atlas named'cort-prob-2mm'

### Additional parameters

In [7]:
params = {
        'reduction':'umap', 
        'clustering':'hdbscan', 
        'min_cluster_size':50, 
        'min_samples':10, 
        'n_neighbors':4, 
        'min_dist':0.0, 
        'n_components':7, 
        'n_clusters':13, 
        'random_state':1111, 
        'cluster_selection_epsilon':0.5, 
        'affinity_reduc' : 'nearest_neighbors',
        'affinity_cluster':'cosine', 
        'linkage':'average',
        'saving_folder': None,
        'plot_name': None
}

In [8]:
view = 'left' #left
kwargs = {
    'surf_mesh': f'pial_{view}', # pial_right, infl_left, infl_right
    'surf_mesh_type': f'pial_{view}',
    'hemi': view, # right
    'view':'lateral', # medial
    'bg_map': f'sulc_{view}', # sulc_right
    'bg_on_data':True,
    'darkness':.8,
    'axes':None, 
    'figure':None
}

### Subjects

In [10]:
subject_names_list = [utils.get_subject_name(sub_id) for sub_id in utils.possible_subjects_id(language)]
subject_ids_list = utils.possible_subjects_id(language)

# UMAP behave strangly when directly apply to the fMRI data of the following subjects, I don't know why. 
# But we still includ them in the study.

black_list_index = [10, 14, 16, 26, 31, 43]
clean_mask = np.ones(len(subject_names_list))
clean_mask[black_list_index] = 0
clean_mask = clean_mask.astype(bool)
clean_subjects = np.array(subject_names_list)[clean_mask]

In [11]:
path_to_beta_maps = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/maps/english/{subject_name}/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf_temporal-shifting-0_{subject_id}_hidden-all-layers/{subject_name}_bert-base-cased_pre-7_1_post-0_norm-None_norm-inf_temporal-shifting-0_{subject_id}_hidden-all-layers_coef_.npy"
mask = "/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/maps/english/{subject_name}/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf_temporal-shifting-0_{subject_id}_hidden-all-layers/{subject_name}_bert-base-cased_pre-7_1_post-0_norm-None_norm-inf_temporal-shifting-0_{subject_id}_hidden-all-layers_Pearson_coeff.nii.gz"

### Words data

In the following cell you can specify the words that you don't want to keep in the clustering analysis.

In [12]:
filtered_words = [
    'a', 
    's',
    'the', 
    'he', 
    'she', 
    'I', 
    'i', 
    'my', 
    'me', 
    '"', 
    ',', 
    '.', 
    '!', 
    'that', 
    'they',
    'this', 
    'them', 
    'and', 
    'you',
    'it',
    '“', 
    '”',
    '‘', 
    '’'
]

In [13]:
# Fetching words
onsets = [pd.read_csv(os.path.join(PROJECT_PATH, 'data', 'onsets-offsets', language, 'word+punctuation_run{index}.csv'.format(index=index))) for index in range(1, 10)]
onsets = [item['word'].values for item in onsets]
onsets = np.hstack(onsets)


In [14]:
# Fetching model activations
all_data = [pd.read_csv(os.path.join(INPUT_PATH, model_name, 'activations_run{index}.csv'.format(index=index))) for index in range(1, 10)]

data_array = [data_tmp.values[:, :768*13] for data_tmp in all_data] # only keep hidden states

# see LayerWiseAnalysis
aggregation_type = 'attention_head' # 'pca', 'layer'
data_agg = reporting.aggregate_beta_maps(
    np.vstack(data_array),
    nb_layers=13, 
    layer_size=768, 
    aggregation_type=aggregation_type
)

In [15]:
# Filtering words
indexes = Parallel(n_jobs=-1)(delayed(reporting.extract_indexes)(onsets, item) for item in filtered_words) #returns a list of list of indexes
mask = np.ones(onsets.shape)
mask[np.hstack([item[0] for item in indexes])] = 0 #flatten 'indexes' object and put 0 at specified indexes values in the mask
mask = mask.astype(bool)

In [16]:
# Applying word mask
data_words_filtered = data_agg[mask, :] # dimension: #words * #features
onsets_filtered = onsets[mask]

In [19]:
standardize_words_activations = True

# standardize words activations if necessary
if standardize_words_activations:
    scaler = StandardScaler(with_mean=True, with_std=True)
    data_words_filtered = scaler.fit_transform(data_words_filtered.T).T

## Data preparation

In [ ]:
all_data = []
all_masks = []
aggregation_type = 'attention_head' # 'pca', 'layer'

for subject_name in tqdm(subjects):
    subject_id = int(subject_name.split('-')[-1])
    path_to_beta_maps = path_to_beta_maps_template.format(model_name=model_name, subject_name=subject_name, subject_id=subject_id)
    mask = create_subject_mask(mask_template, subject_name, subject_id, model_name, global_masker_95, threshold=90)
    
    data = np.load(path_to_beta_maps)
    all_masks.append(mask)
    all_data.append(
        resample_beta_maps(
            aggregate_beta_maps(
                np.vstack(data) if mask is not None else data.copy(), 
                nb_layers=13, 
                layer_size=768, 
                aggregation_type=aggregation_type), 
            original_masker, 
            new_masker)[mask, :])
    #all_original_length.append(data.shape[0] if mask is not None else data.copy().shape[0])

In [ ]:
folder = os.path.join(PROJECT_PATH, 'test/filter_90', model_name)
check_folder(folder)
#np.save(os.path.join(folder, 'all_masks_attention_head.npy'), np.array(all_masks))
#np.save(os.path.join(folder, 'all_data_attention_head.npy'), np.array(all_data))

In [ ]:
average_mask = math_img('img>0.4', img=mean_img(new_masker.inverse_transform(all_masks))) # take the average mask and threshold at 0.5


In [ ]:
plotting.plot_glass_brain(average_mask, colorbar=False, display_mode = 'lzry')
plotting.show()

In [ ]:
voxels_data = np.vstack(all_data)

In [ ]:
standardize_beta_maps = False

# standardize beta maps if necessary
if standardize_beta_maps:
    scaler = StandardScaler(with_mean=True, with_std=True)
    voxels_data = scaler.fit_transform(voxels_data.T).T

## Fonctional clustering

We cluster voxels based on their dependencies towards model features.
Voxels with similar dpeendency patterns are likely to deal with similar information.
To do so, we regroup in one big matrix all subjects voxels as to learn shared clustering patterns.

In [ ]:
nb_voxels=data.shape[0]/len(subject_names_list)

plot_functional_clustering(
    data=voxels_data, 
    mask=np.hstack(all_masks),
    masker=new_masker,
    average_mask=average_mask,
    nb_voxels=nb_voxels,
    nb_subjects=len(subject_names_list),
    data_name='{}_attention-head'.format(model_name),
    params=params,
    saving_folder=SAVING_FOLDER,
    n_neighbors_list=[3, 4, 5],
    min_samples_list=[5, 8, 10, 12, 15, 17, 20, 23,  25, 27, 30],
    min_cluster_size_list=[200, 300, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1100, 1200, 1300, 1350 ,1400, 1450, 1500, 1550, 1600, 1700, 1750, 1800, 1850, 1900],
    tmp_folder=TMP_FOLDER, 
    return_plot=False
    **kwargs
    )

Have a loook at 'SAVING_FOLDER' were all figures are displayed.

Interesting plots:

In [ ]:
paths = [
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1250_n-neigh-5_n-comp-3_aff-clus-euclidean_link-ward_min-samples-3_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-23.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1450_n-neigh-4_n-comp-3_aff-clus-euclidean_link-ward_min-samples-10_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-15.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1500_n-neigh-4_n-comp-3_aff-clus-euclidean_link-ward_min-samples-10_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-15.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1550_n-neigh-4_n-comp-3_aff-clus-euclidean_link-ward_min-samples-10_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-15.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-pca_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1450_n-neigh-4_n-comp-3_aff-clus-euclidean_link-ward_min-samples-5_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-19.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-pca_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1500_n-neigh-4_n-comp-3_aff-clus-euclidean_link-ward_min-samples-5_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-19.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-pca_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1550_n-neigh-4_n-comp-3_aff-clus-euclidean_link-ward_min-samples-5_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-17.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1350_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-26.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1400_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-26.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1400_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-23_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-24.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1400_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-22.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1400_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-29.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1450_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-15_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-29.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1450_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-17_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-27.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1450_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-24.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1450_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-19.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1450_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-28.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1500_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-17_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-25.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1500_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-23.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1500_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-19.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1500_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-25.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1500_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-23.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1550_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-17_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-24.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1550_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-22.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1550_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-19.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1550_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-25.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1550_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-23.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1600_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-21.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1600_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-18.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1600_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-22.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1600_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-22.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1700_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-16.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1700_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-16.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1700_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-20.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1700_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-20.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1750_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-16.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1750_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-20.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1750_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-20.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1800_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-16.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1800_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-19.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1800_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-18.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1850_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-15.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1850_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-17.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1850_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-17.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1900_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-17.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1900_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-17.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1850_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-15.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1900_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-13.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1850_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-44.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1850_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-44.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1870_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-44.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1870_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-23_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-45.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1870_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-44.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1870_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-44.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1870_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-43.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1900_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-44.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1900_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-23_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-45.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1900_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-44.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1900_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-44.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1900_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-42.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1920_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-44.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1920_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-23_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-45.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1920_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-44.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1920_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-43.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1920_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-42.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-950_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-15_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-25.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-900_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-15_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-29.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1400_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-15_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-15.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1350_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-15_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-17.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1300_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-15_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-17.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1200_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-22.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1100_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-23.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1100_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-15_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-21.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1100_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-10_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-29.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1000_n-neigh-3_n-comp-3_aff-clus-cosine_link-average_min-samples-15_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-25.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1200_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-17_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-39.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1200_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-23_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-37.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1750_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-36.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1750_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-28.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1780_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-36.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1780_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-23_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-36.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1780_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-35.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1780_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-27.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1780_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-36.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1800_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-36.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1800_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-23_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-34.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1800_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-35.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1800_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-27.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1800_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-30_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-36.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1830_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-34.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1830_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-23_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-33.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1830_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-25_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-35.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1830_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-26.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1850_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-33.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/bert-base-cased_pre-7_1_post-0_norm-None_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1850_n-neigh-4_n-comp-3_aff-clus-cosine_link-average_min-samples-27_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-26.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-550_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-8_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-20.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-600_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-8_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-18.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-650_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-8_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-15.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-800_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-12_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-15.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-800_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-3_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-16.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-850_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-3_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-16.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-average_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-900_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-3_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-15.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1700_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-10_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-24.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/gpt2_pre-20_1_norm-inf_norm-inf/data-attention_head_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1900_n-neigh-5_n-comp-3_aff-clus-cosine_link-average_min-samples-10_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-21.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-pca_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1550_n-neigh-5_n-comp-3_aff-clus-euclidean_link-ward_min-samples-20_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-7.npy',     
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-pca_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-1550_n-neigh-5_n-comp-3_aff-clus-euclidean_link-ward_min-samples-10_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-9.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-pca_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-650_n-neigh-3_n-comp-3_aff-clus-euclidean_link-ward_min-samples-3_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-60.npy',
 '/neurospin/unicog/protocols/IRMf/LePetitPrince_Pallier_2018/LePetitPrince/derivatives/fMRI/clustering/data-pca_umap_hdbscan/labels/all-subjects_red-umap_clust-hdbscan_min-c-size-650_n-neigh-3_n-comp-3_aff-clus-euclidean_link-ward_min-samples-2_metric-r-cosine_metric-c-euclidean_out-metric-euclidean_nb_clusters-65.npy',
        ]


In [ ]:
nb_voxels=data.shape[0]/len(subject_names_list)

multi_plot(
    paths, 
    saving_path=SAVING_FOLDER,
    nb_subjects=len(subject_names_list),
    nb_voxels=nb_voxels,
    plot_name='', 
    new_masker=new_masker, 
    average_mask=average_mask,
    return_plot=True, 
    **kwargs
    )

## Semantic clustering

We cluster words based on their embedding, and apply the transformations learnt to the voxels of each subject independently.

In [ ]:
reporting.plot_semantic_clustering(
    data_words_filtered,
    path_to_data_template,
    mask_template,
    original_masker,
    new_masker,
    onsets_filtered,
    subject_names_list,
    subject_ids_list,
    data_name='{}_attention-head'.format(model_name),
    params=params,
    saving_folder=SAVING_FOLDER,
    n_neighbors_list=[3, 4, 5],
    min_samples_list=[5, 8, 10, 12, 15, 17, 20, 23,  25, 27, 30],
    min_cluster_size_list=[200, 300, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1100, 1200],
    tmp_folder=TMP_FOLDER, 
    model_name=model_name,
    aggregation_type='attention_head', # 'pca', 'layer'
    return_plot=False,
    standardize_beta_maps=True
    **kwargs
    )

Have a look at scatter text plots that are saved in 'SAVING_FOLDER' (they are too heavy to display, ~10MB each).

In [ ]:
d = extract_clusters(labels_, onsets[mask])